In [1]:
#Import Libraries
import pandas as pd
from pathlib import Path

In [2]:
#Import CSV with all data
data_path = Path("historical_data.csv")
historical_data = pd.read_csv(data_path)
historical_data.rename( columns = {"Unnamed: 0":"date"}, inplace = True)
historical_data.head()

,Date,XLY,XLP,XLE,XLF,XLV,XLI,XLB,XLK,XLU,10 YR
0,2008-01-02,32.13,28.34,79.75,28.41,34.94,38.37,41.33,26.10,41.98,3.91
1,2008-01-03,31.78,28.22,80.21,28.20,35.18,38.58,41.80,26.08,42.08,3.91
2,2008-01-04,30.78,28.04,78.01,27.39,34.83,37.60,40.74,25.07,42.30,3.88
3,2008-01-07,30.93,28.55,77.04,27.55,35.57,37.34,40.25,24.97,43.25,3.86
4,2008-01-08,30.32,28.38,76.08,26.54,35.81,36.39,39.60,24.21,43.33,3.86


In [193]:
# Define a function that returns the indexes where the 10 yr treasury momentum crossed a certain threshold
def spike_selector(df):
    signal1 = 0
    daycount1 = 0
    indexes = []
    df["mmt10"] = (df["10 YR"] - df["10 YR"].shift(10))
    df["roll_10"] = df["mmt10"].rolling(20).mean()
    no_idx = df.reset_index()
    for index, row in zip(no_idx.index, no_idx["roll_10"]):
        if signal1 == 1:
            daycount1 += 1
            if daycount1 > 90:
                signal1 = 0
                daycount1 = 0
        if row >= 0.187 and signal1 == 0:
            #print (signal1, daycount1)
            indexes.append(index)
            signal1 = 1
    df.drop(columns = ["mmt10", "roll_10"], inplace = True)
    return indexes

In [194]:
#Create empty dictionary to store new dataframes.  Then slice 90 days following each treaury yeild increase selected by the above function
yield_spikes_dict = {}
for index in spike_selector(historical_data):
    yield_spikes_dict[index] = pd.DataFrame
for index in spike_selector(historical_data):
    start = index
    end = start + 90
    yield_spikes_dict[index] = historical_data.iloc[start: end]

print(yield_spikes_dict.keys())

dict_keys([269, 360, 498, 736, 1357, 2221, 3292])


In [195]:
print(spike_selector(historical_data))


[269, 360, 498, 736, 1357, 2221, 3292]


In [196]:
#For loop to update keys - DOES NOT WORK
count = 1
final_dict={}
spike_list=[]
for key, value in yield_spikes_dict.items():
    new_df = yield_spikes_dict[key]
    new_df.reset_index(inplace=True)
    new_df.drop(columns=['Date','index'], axis=1, inplace=True)
    if new_df["10 YR"].count()>=85:
        final_dict[str("spike_")+str(count)]=new_df
        count +=1
    
print(final_dict.keys())

dict_keys(['spike_1', 'spike_2', 'spike_3', 'spike_4', 'spike_5', 'spike_6'])


In [197]:
#Updating keys manually
# yield_spikes_dict["spike_1"] = yield_spikes_dict.pop(269)
# yield_spikes_dict["spike_2"] = yield_spikes_dict.pop(360)
# yield_spikes_dict["spike_3"] = yield_spikes_dict.pop(498)
# yield_spikes_dict["spike_4"] = yield_spikes_dict.pop(736)
# yield_spikes_dict["spike_5"] = yield_spikes_dict.pop(1357)
# yield_spikes_dict["spike_6"] = yield_spikes_dict.pop(2221)
# yield_spikes_dict["spike_7"] = yield_spikes_dict.pop(3292)

In [198]:
final_dict["spike_5"]
spikes_df= pd.concat(final_dict.values(), axis = 1, keys = final_dict.keys())

In [200]:
spikes_df.head()

spike_1                                                                ...  \
      XLY    XLP    XLE   XLF    XLV     XLI    XLB    XLK    XLU 10 YR  ...   
0   19.81  22.83  47.54  9.49  26.53  16.952  21.92  15.12  29.61  2.87  ...   
1   19.28  22.11  46.89  9.28  26.20  16.590  21.06  14.74  28.93  2.87  ...   
2   19.11  21.96  46.15  9.25  26.45  16.211  20.77  14.96  29.07  2.76  ...   
3   19.66  22.43  46.97  9.03  27.08  16.509  21.15  15.22  29.30  2.89  ...   
4   19.21  21.83  47.29  8.95  26.91  16.421  21.50  15.21  29.53  2.95  ...   

  spike_6                                                                    
      XLY     XLP      XLE    XLF    XLV    XLI    XLB    XLK     XLU 10 YR  
0   81.68  50.850  72.8100  22.24  69.91  59.61  48.89  47.85  46.780  2.33  
1   82.68  51.100  72.7650  22.25  68.95  59.88  49.19  47.98  46.940  2.31  
2   82.76  50.810  73.1200  22.38  69.20  60.32  49.43  47.79  46.495  2.36  
3   82.97  51.205  72.7300  22.43  69.46  60.65  49.58  47.99  47.155  2.36  
4   82.33  51.320  71.6901  22.13  68.92  60.23  49.37  48.03  48.085  2.32  

[5 rows x 60 columns]